In [220]:
from pymongo import MongoClient
import pprint
import urllib.request

import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

# Requests sends and recieves HTTP requests.
import requests

# Beautiful Soup parses HTML documents in python.
from bs4 import BeautifulSoup

import json
import time
import copy 

In [346]:
#pull census variables 
#storing census variable names(unreadable) with readable column names 
var_req = requests.get('https://api.census.gov/data/2018/acs/acs1/spp/variables.json').json()


col_name = {}
variables = var_req['variables']

for k, v in variables.items():
    col_name[k] = v['label']
    
#still need to clean up the column names and remove !! and other unnecssary data. 

In [347]:
#remove all the columns that have PR in them
clean_col_name = col_name.copy()
stat = 'PR'


for k, v in col_name.items():
    if stat in k:
        del clean_col_name[k]

In [381]:
#flexible pull request for api call, generates names for api url
def census_name_pull(name_dict, word_search):
    name_list = ['NAME','GEO_ID']


    for k, v in name_dict.items():
        if word_search in v:
            name_list.append(k)
    return sorted(name_list)

In [382]:
#pull columns for sex & age by county in state
sex_age_pull = census_name_pull(clean_col_name,'SEX AND AGE')


In [383]:
#pull columns for employment status by county in state
emp_stat_pull = census_name_pull(clean_col_name,'EMPLOYMENT STATUS')


In [384]:
#pull columns for industry by county in state
industry_pull = census_name_pull(clean_col_name,'INDUSTRY')


In [385]:
#pull columns for income by county in state
income_pull = census_name_pull(clean_col_name,'INCOME')


In [388]:
#connect to api_url for census per 
get_vars = [sex_age_pull, emp_stat_pull, industry_pull, income_pull]

def api_pull_request(get_vars):

    api_key = 'ce30285f0f2fedc096b5bf054c7d573ea6558309'
    base_url = 'https://api.census.gov/data/2018/acs/acs1/spp?'

    predicates = {}
    predicates['get'] = ','.join(get_vars)
    predicates['for'] = 'county:*'
    predicates['in'] = 'state:*'
    predicates['key'] = api_key
    
    r = requests.get(base_url, params=predicates)
    
    return r.json()

# api_url = '''https://api.census.gov/data/2018/acs/acs1/spp?get=NAME,GEO_ID,S0201_001E,S0201_006E,S0201_213E,S0201_194E&for=county:*&in=state:*&key=ce30285f0f2fedc096b5bf054c7d573ea6558309'''

In [394]:
#created sex and age pull request and dataframe
sex_age_data = api_pull_request(sex_age_pull)
sex_age_df = pd.DataFrame(sex_age_data)


In [395]:
#created employee status pull request and dataframe
emp_stat_data = api_pull_request(emp_stat_pull)
emp_stat_df = pd.DataFrame(emp_stat_data)


In [396]:
#created income pull request and dataframe
income_data = api_pull_request(income_pull)
income_df = pd.DataFrame(income_data)

In [397]:
#created industry pull request and dataframe
industry_data = api_pull_request(industry_pull)
industry_df = pd.DataFrame(industry_data)